# Figures for Frame et al. 2022 "On mass conservation..."

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import xarray as xr
import copy
import os
import sys
import metrics
import random
import scipy.stats as st
import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import mutual_info_score

In [2]:
def cm2inch(width: float, height: float) -> tuple:
    """Converts figure size from centimeters to inch.
    
    Parameters
    ----------
    width : float
        Width of figure in centimeters
    height : float
        Height of figure in centimeters
    
    Returns
    -------
    Tuple[float, float]
        Tuple containing width and height in inches
    """
    inch = 2.54
    return (width / inch, height / inch)

In [3]:
# Convert flow to   CFS mm -> ft     km^2 -> ft^2    hr->s
conversion_factor = 0.00328084 * 10763910.41671 / 3600 / 24

In [4]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [5]:
# Camels attributes with RI information
dataName = '../data/camels_attributes.csv'
# load the data with pandas
pd_attributes = pd.read_csv(dataName, sep=',', index_col='gauge_id')

# Add the basin ID as a 8 element string with a leading zero if neccessary
basin_id_str = []
for a in pd_attributes.index.values:
    basin_id_str.append(str(a).zfill(8))
pd_attributes['basin_id_str'] = basin_id_str

In [6]:
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE']
loop_these_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE', 'mi']

In [7]:
def calc_MI(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

def calculate_all_metrics_for_frequency_analysis(analysis_dict, flows):
    
    sims = list(flows.keys())[:-1]

    for metric in loop_these_metrics:

        score = {sim:0 for sim in sims}
        
        if metric == 'NSE':
            for sim in sims:
                score[sim] = metrics.nse(flows['obs'],flows[sim])
        if metric == 'MSE':
            for sim in sims:
                score[sim] = metrics.mse(flows['obs'],flows[sim])
        if metric == 'RMSE':
            for sim in sims:
                 score[sim] = metrics.rmse(flows['obs'],flows[sim])
        if metric == 'KGE':
            for sim in sims:
                score[sim] = metrics.kge(flows['obs'],flows[sim])
        if metric == 'KGEss':
            for sim in sims:
                score[sim] = (metrics.kge(flows['obs'],flows[sim])+0.41421356237)/np.sqrt(2)
        if metric == 'Alpha-NSE':
            for sim in sims:
                score[sim] = metrics.alpha_nse(flows['obs'],flows[sim])
        if metric == 'Beta-NSE':
            for sim in sims:
                score[sim] = metrics.beta_nse(flows['obs'],flows[sim])
        if metric == 'Pearson-r':
            for sim in sims:
                score[sim] = metrics.pearsonr(flows['obs'],flows[sim])
        if metric == 'Peak-Timing':
            for sim in sims:
                score[sim] = np.abs(metrics.mean_peak_timing(flows['obs'],flows[sim]))
        if metric == 'FHV':
            for sim in sims:
                score[sim] = metrics.fdc_fhv(flows['obs'],flows[sim])          
        if metric == 'FLV':
            for sim in sims:
                score[sim] = metrics.fdc_flv(flows['obs'],flows[sim])          
        if metric == 'FMS':
            for sim in sims:
                score[sim] = metrics.fdc_fms(flows['obs'],flows[sim])   
                
        if metric == "mi":
            for sim in sims:
                score[sim] = calc_MI(flows['obs'],flows[sim], 100)

          
        for sim in sims:
            analysis_dict[metric][sim].append(score[sim])
        
    return

# ---------------------------------
# Standard Time Split

test period(1989-1999) is the same period used by previous studies  which allows us to confirm that the deep learning models (LSTM andMC-LSTM) trained for this project perform as expected relative to prior work. 
These metrics are broadly equivalent to thosereported for single models (not ensembles) by (Kratzert et al., 2019c) (LSTM) and (Hoedt et al., 2021) (MC-LSTM)

# Standard Time Split
# ---------------------------------

In [8]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac']
flows =         ['lstm','mc', 'sac', 'obs']
#-------------------------------------------------------------------------------------------------

calculate_metrics_now = False
if calculate_metrics_now:

    lstm_results_time_split1   = {}
    mclstm_results_time_split1 = {}
    sacsma_results_time_split1 = {}
    
    for forcing_type in ['nldas', 'daymet']:

        with open('./model_output_for_analysis/lstm_time_split1_{}_ens.p'.format(forcing_type), 'rb') as fb:
            lstm_results_time_split1[forcing_type] = pkl.load(fb)
        with open('./model_output_for_analysis/mclstm_time_split1_{}_ens.p'.format(forcing_type), 'rb') as fb:
            mclstm_results_time_split1[forcing_type] = pkl.load(fb)
        with open('./model_output_for_analysis/sacsma_time_split1_{}_ens.p'.format(forcing_type), 'rb') as fb:
            sacsma_results_time_split1[forcing_type] = pkl.load(fb)

    basin_list = lstm_results_time_split1[forcing_type].keys()

    st1_dict = {}
    st1_dict_stackz = {forcing_type:{flow:[] for flow in flows} for forcing_type in ['nldas', 'daymet']}

    for forcing_type in ['nldas', 'daymet']:

        analysis_dict_temp_large = {}
        #-------------------------------------------------------------------------------------------------            

        #-------------------------------------------------------------------------------------------------
        #-----LOOP THROUGH BASINS------------------------------------------------------------------------
        #-------------------------------------------------------------------------------------------------

        for ib, basin_0str in enumerate(basin_list): 
            basin_int = int(basin_0str)



            #-------------------------------------------------------------------------------------------------
            # Setting up the dictionary for the single basin results. Then will add to the overall dict.
            analysis_dict_temp_small = {metric:{model:[] for model in models} for metric in loop_these_metrics}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # We need the basin area to convert to CFS, to interpolate the RI from LPIII
            basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
            basin_str = str(basin_int).zfill(8)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            #----  Set the time period for metrics   ---------------------------------------------------------
            date_from = '1989-10'
            date_to = '1999-09'
            #-------------------------------------------------------------------------------------------------    

            #-------------------------------------------------------------------------------------------------
            # Make dictionary with all the flows
            flow_mm = {}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Standard LSTM data trained on all years
            xrr = lstm_results_time_split1[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.datetime.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------        
            # Mass Conserving LSTM data
            xrr = mclstm_results_time_split1[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.datetime.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # SACSMA Sinlge run
            df = sacsma_results_time_split1[forcing_type][basin_0str]
            flow_mm['sac'] = df.loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # OBSERVATIONS
            xrr = mclstm_results_time_split1[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_obs']
            flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.datetime.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------

            for flow in flows:
                st1_dict_stackz[forcing_type][flow].extend(list(flow_mm[flow].values))

            #-------------------------------------------------------------------------------------------------
            # Make all xarray data similar
            for iflow in flows:
                if iflow == 'nwm': #already in the correct format
                    continue
                if iflow == 'sac': #already in the correct format
                    flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
                else:
                    flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            calculate_all_metrics_for_frequency_analysis(analysis_dict_temp_small, flow_mm)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            #Now that the basin has been analyzed successfully, add it to the larger dictionary
            analysis_dict_temp_large[basin_0str] = analysis_dict_temp_small
            #------------------------------------------------------------------------------------------------- 

        st1_dict[forcing_type] = {metric:{model:[] for model in models} for metric in loop_these_metrics}
        for ib, basin_0str in enumerate(basin_list):
            try:
                for metric in loop_these_metrics:
                    for model in models:
                        st1_dict[forcing_type][metric][model].extend(analysis_dict_temp_large[basin_0str][metric][model])
            except:
                continue
                
    with open("statistics_table_st1.pkl", 'wb') as fb:
        pkl.dump(st1_dict, fb)
        
    with open("st1_flows_stacked.pkl", 'wb') as fb:
        pkl.dump(st1_dict_stackz, fb)
        
else:
        
    with open('statistics_table_st1.pkl', 'rb') as fb:
        st1_dict = pkl.load(fb)
        
    with open('st1_flows_stacked.pkl', 'rb') as fb:
        st1_dict_stackz = pkl.load(fb)

In [9]:
# Included in the text, but not in a table or figure.
for forcing_type in ['nldas', 'daymet']:
    mi_lstm = calc_MI(np.array(st1_dict_stackz[forcing_type]['obs'])[:,0], 
                      np.array(st1_dict_stackz[forcing_type]['lstm'])[:,0], 100)
    mi_mc = calc_MI(np.array(st1_dict_stackz[forcing_type]['obs'])[:,0], 
                    np.array(st1_dict_stackz[forcing_type]['mc'])[:,0], 100)
    mi_sac = calc_MI(np.array(st1_dict_stackz[forcing_type]['obs'])[:,0], 
                     np.array(st1_dict_stackz[forcing_type]['sac']), 100)
    print("Mutual information LSTM", np.round(mi_lstm,2))
    print("Mutual information MC", np.round(mi_mc,2))
    print("Mutual information SAC-SMA", np.round(mi_sac,2))

Mutual information LSTM 0.39
Mutual information MC 0.37
Mutual information SAC-SMA 0.34
Mutual information LSTM 0.4
Mutual information MC 0.37
Mutual information SAC-SMA 0.33


In [10]:
round_to = 2
for metric in table_metrics:
    
    printstuf = {forcing_type:{model:{} for model in ['lstm', 'mc','sac']} for forcing_type in ['daymet','nldas']}

    for forcing_type in ['daymet','nldas']:
        for model in ['lstm', 'mc','sac']:
            data=st1_dict[forcing_type][metric][model]
            printstuf[forcing_type][model]['mean']=np.round(np.nanmedian(data),round_to)
            if np.abs(st.t.interval(alpha=0.95, df=len(data)-1,loc=0,scale=st.sem(data))[0]) < \
                np.abs(printstuf[forcing_type][model]['mean']):
                conf = np.round(st.t.interval(alpha=0.95, df=len(data)-1, loc=0,scale=st.sem(data))[0],round_to)
            else:
                conf = 'n/a'
            printstuf[forcing_type][model]['conf']=conf
    
    
    print('{}  &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm{}$ \\'.format(metric,
                      printstuf['daymet']['lstm']['mean'],printstuf['daymet']['lstm']['conf'],
                                                                               
                      printstuf['daymet']['mc']['mean'],printstuf['daymet']['mc']['conf'],
                                                                               
                      printstuf['daymet']['sac']['mean'],printstuf['daymet']['sac']['conf'],
                                                
                      printstuf['nldas']['lstm']['mean'],printstuf['nldas']['lstm']['conf'],
                      
                      printstuf['nldas']['mc']['mean'],printstuf['nldas']['mc']['conf'],
                                                                               
                      printstuf['nldas']['sac']['mean'],printstuf['nldas']['sac']['conf']))
                                            
printstuf = {forcing_type:{model:{} for model in ['lstm', 'mc','sac']} for forcing_type in ['daymet','nldas']}
for forcing_type in ['daymet','nldas']:
    for model in ['lstm', 'mc','sac']:
        data=st1_dict[forcing_type]['Peak-Timing'][model]
        printstuf[forcing_type][model]['mean']=np.round(np.nanmedian(data),round_to)
        printstuf[forcing_type][model]['conf']=np.round(st.t.interval(alpha=0.95, 
                                                             df=len(data)-1,
                                                             loc=0,scale=st.sem(data))[0],round_to)
print('Peak-Timing &{} $\pm{}$ &{} $\pm{}$ &{} $\pm{}$ &{} $\pm{}$ &{} $\pm${} &{} $\pm$ {}\\'.format(
                      printstuf['daymet']['lstm']['mean'],printstuf['daymet']['lstm']['conf'],
                                                                               
                      printstuf['daymet']['mc']['mean'],printstuf['daymet']['mc']['conf'],
                                                                               
                      printstuf['daymet']['sac']['mean'],printstuf['daymet']['sac']['conf'],
                                                
                      printstuf['nldas']['lstm']['mean'],printstuf['nldas']['lstm']['conf'],
                      
                      printstuf['nldas']['mc']['mean'],printstuf['nldas']['mc']['conf'],
                                                                               
                      printstuf['nldas']['sac']['mean'],printstuf['nldas']['sac']['conf']))

NSE  &0.77 $\pm$ -0.02 &0.76 $\pm$ -0.01 &0.65 $\pm$ -0.03 &0.74 $\pm$ -0.01 &0.74 $\pm$ -0.01 &0.67 $\pm-0.02$ \
KGE  &0.76 $\pm$ -0.02 &0.76 $\pm$ -0.02 &0.59 $\pm$ n/a &0.74 $\pm$ -0.02 &0.74 $\pm$ -0.02 &0.68 $\pm-0.02$ \
Pearson-r  &0.89 $\pm$ -0.01 &0.88 $\pm$ -0.01 &0.83 $\pm$ n/a &0.88 $\pm$ -0.01 &0.87 $\pm$ -0.01 &0.83 $\pm-0.01$ \
Alpha-NSE  &0.85 $\pm$ -0.01 &0.84 $\pm$ -0.01 &0.76 $\pm$ -0.02 &0.81 $\pm$ -0.02 &0.81 $\pm$ -0.02 &0.78 $\pm-0.02$ \
Beta-NSE  &-0.04 $\pm$ -0.01 &-0.03 $\pm$ -0.01 &0.06 $\pm$ -0.01 &-0.03 $\pm$ -0.01 &-0.02 $\pm$ -0.01 &-0.01 $\pm-0.01$ \
Peak-Timing &0.3 $\pm-0.03$ &0.3 $\pm-0.03$ &0.38 $\pm-0.06$ &0.32 $\pm-0.03$ &0.31 $\pm$-0.03 &0.41 $\pm$ -0.06\


# ---------------------------------
# NWM Time Split

The second test period (1995-2014) allows us to benchmark against the NWM-Rv2,  which does not provide data prior to2851995. Most of these scores are broadly  equivalent to the metrics for the same models reported for the test period 1989-1999, with the exception of the FHV (high flow bias), FLV (low flow bias), add FMS (flow duration curve bias).  These metrics dependheavily on the observed flow characteristics during a particular test period and,  because they are less stable, are somewhat lessuseful in terms of drawing general conclusions.  We report them here primarily for continuity with previous studies (Kratzertet al., 2019c, b, 2020;  Frame et al., 2020; Nearing et al., 2020b; Klotz et al., 2021; Gauch et al., 2021a), and one of the  objectives of this paper (Section 2.4 is to expand on the high flow (FHV) analysis specifically

# NWM Time Split
# ---------------------------------

In [11]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
loop_these_metrics = metrics.get_available_metrics()
models =        ['nwm', 'lstm','mc', 'sac']
flows =         ['nwm', 'lstm','mc', 'sac', 'obs']
#-------------------------------------------------------------------------------------------------

calculate_metrics_now = False
if calculate_metrics_now:
    lstm_results_time_split2 = {}
    mclstm_results_time_split2 = {}
    sacsma_results_time_split2 = {}
    for forcing_type in ['nldas', 'daymet']:
        with open('./model_output_for_analysis/lstm_time_split2_{}.p'.format(forcing_type), 'rb') as fb:
            lstm_results_time_split2[forcing_type] = pkl.load(fb)
        with open('./model_output_for_analysis/mclstm_time_split2_{}.p'.format(forcing_type), 'rb') as fb:
            mclstm_results_time_split2[forcing_type] = pkl.load(fb)
        with open('./model_output_for_analysis/sacsma_time_split2_{}.p'.format(forcing_type), 'rb') as fb:
            sacsma_results_time_split2[forcing_type] = pkl.load(fb)

    with open('./model_output_for_analysis/nwm_chrt_v2_1d_local.p', 'rb') as fb:
        nwm_results = pkl.load(fb)

    basin_list = sacsma_results_time_split2[forcing_type].keys()

    st2_dict = {}

    for forcing_type in ['nldas', 'daymet']:

        analysis_dict_temp_large = {}
        #-------------------------------------------------------------------------------------------------            

        #-------------------------------------------------------------------------------------------------
        #-----LOOP THROUGH BASINS------------------------------------------------------------------------
        #-------------------------------------------------------------------------------------------------

        for ib, basin_0str in enumerate(basin_list): 
            basin_int = int(basin_0str)


            #-------------------------------------------------------------------------------------------------
            # Get the NWM data for this basin in an xarray dataset.
            xr_nwm = xr.DataArray(nwm_results[basin_0str]['streamflow'].values, 
                     coords=[nwm_results[basin_0str]['streamflow'].index], 
                     dims=['datetime'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Setting up the dictionary for the single basin results. Then will add to the overall dict.
            analysis_dict_temp_small = {metric:{model:[] for model in models} for metric in loop_these_metrics}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # We need the basin area to convert to CFS, to interpolate the RI from LPIII
            basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
            basin_str = str(basin_int).zfill(8)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            #----  Set the time period for metrics   ---------------------------------------------------------
            date_from = '1996-10'
            date_to = '2014-09'
            #-------------------------------------------------------------------------------------------------    

            #-------------------------------------------------------------------------------------------------
            # Make dictionary with all the flows
            flow_mm = {}
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------        
             # NWM data
            sim_nwm = xr_nwm.loc[date_from:date_to]
            # convert from CFS to mm/day
            # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
            flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
            #-------------------------------------------------------------------------------------------------
            # Standard LSTM data trained on all years
            xrr = lstm_results_time_split2[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------        
            # Mass Conserving LSTM data
            xrr = mclstm_results_time_split2[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_sim']
            flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # SACSMA Sinlge run
            df = sacsma_results_time_split2[forcing_type][basin_0str]
            flow_mm['sac'] = df.loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------
            # OBSERVATIONS
            xrr = mclstm_results_time_split2[forcing_type][basin_0str]['1D']['xr']['QObs(mm/d)_obs']
            flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            # Make all xarray data similar
            for iflow in flows:
                if iflow == 'nwm': #already in the correct format
                    continue
                if iflow == 'sac': #already in the correct format
                    flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
                else:
                    flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                                   coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            calculate_all_metrics_for_frequency_analysis(analysis_dict_temp_small, flow_mm)
            #-------------------------------------------------------------------------------------------------


            #-------------------------------------------------------------------------------------------------
            #Now that the basin has been analyzed successfully, add it to the larger dictionary
            analysis_dict_temp_large[basin_0str] = analysis_dict_temp_small
            #------------------------------------------------------------------------------------------------- 

        st2_dict[forcing_type] = {metric:{model:[] for model in models} for metric in loop_these_metrics}
        for ib, basin_0str in enumerate(basin_list):
            try:
                for metric in loop_these_metrics:
                    for model in models:
                        st2_dict[forcing_type][metric][model].extend(analysis_dict_temp_large[basin_0str][metric][model])
            except:
                continue

    with open("statistics_table_st2.pkl", 'wb') as fb:
        pkl.dump(st2_dict, fb)
        
else:
    
    with open('statistics_table_st2.pkl', 'rb') as fb:
        st2_dict = pkl.load(fb)

In [12]:
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE']#,'FHV','FLV','FMS']
round_to = 2
for metric in table_metrics:
    
    printstuf = {forcing_type:{model:{} for model in ['lstm', 'mc','sac', 'nwm']} for forcing_type in ['daymet','nldas']}

    for forcing_type in ['daymet','nldas']:
        if forcing_type == 'daymet':
            modelz = ['lstm', 'mc','sac']
        if forcing_type == 'nldas':
            modelz = ['lstm', 'mc','sac', 'nwm']
        for model in modelz:
            data=st2_dict[forcing_type][metric][model]
            printstuf[forcing_type][model]['mean']=np.round(np.nanmedian(data),round_to)
            if np.abs(st.t.interval(alpha=0.95, df=len(data)-1,loc=0,scale=st.sem(data))[0]) < \
                np.abs(printstuf[forcing_type][model]['mean']):
                conf = np.round(st.t.interval(alpha=0.95, df=len(data)-1, loc=0,scale=st.sem(data))[0],round_to)
            else:
                conf = 'n/a'
            printstuf[forcing_type][model]['conf']=conf
    
    
    print('{}  &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} &{} $\pm$ {} \\'.format(metric,
                      printstuf['daymet']['lstm']['mean'],printstuf['daymet']['lstm']['conf'],
                                                                               
                      printstuf['daymet']['mc']['mean'],printstuf['daymet']['mc']['conf'],
                                                                               
                      printstuf['daymet']['sac']['mean'],printstuf['daymet']['sac']['conf'],
                                                
                      printstuf['nldas']['lstm']['mean'],printstuf['nldas']['lstm']['conf'],
                      
                      printstuf['nldas']['mc']['mean'],printstuf['nldas']['mc']['conf'],
                                                                               
                      printstuf['nldas']['sac']['mean'],printstuf['nldas']['sac']['conf'],
                    
                      printstuf['nldas']['nwm']['mean'],printstuf['nldas']['nwm']['conf']))
                                            
printstuf = {forcing_type:{model:{} for model in ['lstm', 'mc','sac', 'nwm']} for forcing_type in ['daymet','nldas']}
for forcing_type in ['daymet','nldas']:
    if forcing_type == 'daymet':
        modelz = ['lstm', 'mc','sac']
    if forcing_type == 'nldas':
        modelz = ['lstm', 'mc','sac', 'nwm']
    for model in modelz:
        data=st2_dict[forcing_type]['Peak-Timing'][model]
        printstuf[forcing_type][model]['mean']=np.round(np.nanmedian(data),round_to)
        printstuf[forcing_type][model]['conf']=np.round(st.t.interval(alpha=0.95, 
                                                             df=len(data)-1,
                                                             loc=0,scale=st.sem(data))[0],round_to)
print('Peak-Timing &{} $\pm{}$ &{} $\pm{}$ &{} $\pm{}$ &{} $\pm$ {} &{} $\pm${} &{} $\pm$ {} &{} $\pm$ {}\\'.format(
                      printstuf['daymet']['lstm']['mean'],printstuf['daymet']['lstm']['conf'],
                                                                               
                      printstuf['daymet']['mc']['mean'],printstuf['daymet']['mc']['conf'],
                                                                               
                      printstuf['daymet']['sac']['mean'],printstuf['daymet']['sac']['conf'],
                                                
                      printstuf['nldas']['lstm']['mean'],printstuf['nldas']['lstm']['conf'],
                      
                      printstuf['nldas']['mc']['mean'],printstuf['nldas']['mc']['conf'],
                                                                               
                      printstuf['nldas']['sac']['mean'],printstuf['nldas']['sac']['conf'],

                      printstuf['nldas']['nwm']['mean'],printstuf['nldas']['nwm']['conf']))

NSE  &0.74 $\pm$ -0.02 &0.74 $\pm$ -0.02 &0.59 $\pm$ -0.08 &0.71 $\pm$ -0.05 &0.72 $\pm$ -0.02 &0.63 $\pm$ -0.05 &0.63 $\pm$ -0.05 \
KGE  &0.78 $\pm$ -0.02 &0.77 $\pm$ -0.02 &0.56 $\pm$ n/a &0.77 $\pm$ -0.02 &0.74 $\pm$ -0.02 &0.68 $\pm$ -0.02 &0.67 $\pm$ -0.05 \
Pearson-r  &0.88 $\pm$ -0.01 &0.88 $\pm$ -0.01 &0.81 $\pm$ n/a &0.86 $\pm$ -0.01 &0.86 $\pm$ -0.01 &0.81 $\pm$ -0.01 &0.82 $\pm$ -0.01 \
Alpha-NSE  &0.96 $\pm$ -0.02 &0.91 $\pm$ -0.01 &0.88 $\pm$ -0.02 &0.94 $\pm$ -0.02 &0.87 $\pm$ -0.02 &0.83 $\pm$ -0.02 &0.85 $\pm$ -0.03 \
Beta-NSE  &0.03 $\pm$ -0.01 &0.03 $\pm$ -0.01 &0.13 $\pm$ -0.02 &0.01 $\pm$ -0.01 &-0.01 $\pm$ -0.01 &-0.01 $\pm$ n/a &-0.01 $\pm$ n/a \
Peak-Timing &0.34 $\pm-0.03$ &0.33 $\pm-0.03$ &0.45 $\pm-0.06$ &0.38 $\pm$ -0.03 &0.4 $\pm$-0.03 &0.53 $\pm$ -0.06 &0.54 $\pm$ -0.05\
